Import necessary packages

In [8]:
import pandas as pd
import tweepy
from datetime import datetime
import time

Function to creating list all_tweets for a  specific keyword

In [9]:
# Helper Function to fetch tweets (built into create_tweet_list)
def get_all_tweets(topic):

    # Importing keys for Twitter API
    log = pd.read_csv('C:/Users/Marc/Dropbox/06_ESCP/01_Uni/06_MA Thesis/04_Code/01_Input/twitter_input_keys.csv', sep= ';', index_col = 'Name')

    # Selecting keys
    BEARER_TOKEN = log['key']['Bearer token']
    consumer_key = log['key']['API key']
    consumer_secret = log['key']['API secretkey']
    access_token = log['key']['AccessToken']
    access_token_secret = log['key']['AccessToken Secret']

    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)

    auth.set_access_token(access_token, access_token_secret)

    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

    # Define Search parameters for Tweepy
    # Count setting
    new_tweets = api.search(q = topic, lang = 'en', count = 100, truncated = False, tweet_mode = 'extended', result_type = 'recent')


    # Store in JSON format
    json_data = [t._json for t in new_tweets]

    # Transform data from JSON to Pandas DF
    df = pd.json_normalize(json_data)

    # Select columns necessary for sentiment analysis
    all_tweets = df[['id','created_at','full_text']]
    all_tweets.head(1)
    oldest = all_tweets.id.iloc[-1] - 1

    # While loop to mine more then the 200 tweets max at once
    while len(all_tweets) < 200:

        print(f"getting tweets before {oldest}")

        new_tweets = api.search(q = 'Lakers', lang = 'en', count = 100, truncated = False, tweet_mode = 'extended', result_type = 'recent', max_id = oldest)


        json_data = [t._json for t in new_tweets]


        df = pd.json_normalize(json_data)
        df2 = df[['id','created_at','full_text']]

        all_tweets = all_tweets.append(df2, ignore_index= True)

        all_tweets.reset_index()
        all_tweets

        oldest = all_tweets.id.iloc[-1]

    print(f'The list is {len(all_tweets)} entires long')
    return all_tweets

In [10]:
# Function to fetch tweet for multiple search words
def create_tweet_list(all_tweets):
    # looping through the words and getting the respective neewest tweets
    tweets  = pd.DataFrame(get_all_tweets(all_tweets[0]))

    for i in range(1, len(all_tweets)):
        new = get_all_tweets(all_tweets[i])
        new_df = pd.DataFrame(new)
        tweets = tweets.append(new_df, ignore_index = True)
    return tweets

In [11]:
# input list of tweets to clean the date format
def clean_created_at(tweet_list):
    # remove duplicates in case different searchword lead to the same tweets
    tweet_list.drop_duplicates(subset = ['id'])
    tweet_list.drop_duplicates(subset = ['full_text'])


    # to Datetime to have +0000 in the end
    tweet_list['created_at'] = pd.to_datetime(tweet_list['created_at'])

    # Revert to string to be able to perform following action (remove +00:00 --> indicating UTC)
    tweet_list['created_at'] = [str(x) for x in tweet_list['created_at']]

    # Removing +00:00
    tweet_list['created_at'] = [x[:len(x) - 6] for x in tweet_list['created_at']]

    # Reversing to datetime format
    tweet_list['created_at'] = pd.to_datetime(tweet_list['created_at'])

    return tweet_list

# Current Working point
below

Create Df with all the hashtags

In [12]:
# Four very common search terms in relation to the teams on twitter - Dictionary created for lookup
hashtags = {}
hashtags['LAL'] = ['Lakers','lakers nation','lakers basketball', 'Lakers news']
hashtags['GSW'] = ['Golden State Warriors', 'golden state warriors basketball', 'golden state warriors fan', 'warriors nation']
hashtags['CHB'] = ['Chicago Bulls', 'bulls', 'chicago bulls nation', 'chicago bulls fan' ]
hashtags['BOC'] = ['Boston Celtics', 'celtics', 'celtics nation', 'celtics pride']
hashtags['MIH'] = ['heat nation','Miami Heat', 'miami heat fan', 'heat basketball' ]

In [13]:
# key to iterate over - look up search terms in dictionary
keys = ['LAL','GSW','MIH','BOC','CHB']

In [14]:
error_log = []
c = 0
while c <= 8:
    for i in keys:
        for j in hashtags[i]:
            try:
                tweets = create_tweet_list((hashtags[i]))
                tweets_cleaned = clean_created_at(tweets)
                df_temp = pd.read_excel('C:/Users/Marc/Dropbox/06_ESCP/01_Uni/06_MA Thesis/04_Code/02_Output/02_Tweets/' + i + '/' + i + '_Tweets.xlsx')
                df_temp = df_temp.append(tweets_cleaned)
                df_temp.to_excel('C:/Users/Marc/Dropbox/06_ESCP/01_Uni/06_MA Thesis/04_Code/02_Output/02_Tweets/' + i + '/' + i + '_Tweets.xlsx', index = False)
                df_temp.to_excel('C:/Users/Marc/Dropbox/06_ESCP/01_Uni/06_MA Thesis/04_Code/02_Output/02_Tweets/' + i + '/01_Per Iteration/' + i + str(c) + '_Tweets_backup.xlsx', index = False)
                print('start sleeping')
                print(datetime.now().strftime("%Y/%m/%d %H:%M:%S"))
                time.sleep(450)
                print(c)
                print('done sleeping')
                print(datetime.now().strftime("%Y/%m/%d %H:%M:%S"))
            except KeyError:

                print(str(i) + '##########key error in ' + str(c))
            finally:
                pass
    c += 1





getting tweets before 1397888542710321153
The list is 200 entires long
getting tweets before 1397445449242619906
The list is 200 entires long
getting tweets before 1397821838449733632
The list is 200 entires long
getting tweets before 1397704171269378047
The list is 200 entires long
start sleeping
2021/05/27 14:26:08
0
done sleeping
2021/05/27 14:33:38
getting tweets before 1397891287819472900
The list is 200 entires long
getting tweets before 1397445449242619906
The list is 200 entires long
getting tweets before 1397827750958936064
The list is 200 entires long
getting tweets before 1397710782176002052
The list is 200 entires long
start sleeping
2021/05/27 14:37:13
0
done sleeping
2021/05/27 14:44:43
getting tweets before 1397894294845931521
The list is 200 entires long
getting tweets before 1397445449242619906
The list is 200 entires long
getting tweets before 1397833455468175359
The list is 200 entires long
getting tweets before 1397716972314533890
The list is 200 entires long
start 

TweepError: Twitter error response: status code = 503

Timestamp is shown in UTC, not time of location of the IP (e.g. Germany which is UTC+2 (Summer Time))

